# Introduction to encoding in CKKS

CKKS exploits the rich structure of polynomial rings for its plaintext and ciphertext spaces. Nonetheless, data comes more often in the form of vectors than polynomials.

We will denote by $N$ the degree of our polynomial degree modulus, which will be a power of 2. We denote by $\Phi_M(X) = X^N + 1$ the $m$-th [cyclotomic polynomial](https://en.wikipedia.org/wiki/Cyclotomic_polynomial) (note that $M=2N$). The plaintext space will be the polynomial ring $\mathcal{R} = \mathbb{Z}[X]/(X^N + 1)$. Let us denote by $\xi_M$, the $m$-th root of unity : $\xi_M = e^{2 i \pi / M}$.

Therefore it becomes necessary to encode our input $z \in \mathbb{C}^{N/2}$, into a polynomial $m(X) \in \mathbb{Z}[X]/(X^N + 1)$. 

Conversely, once computation has been done and we obtain a plaintext polynomial $m(X)$, we need to decode it to obtain the results of the computations. Therefore we also need to find a way to decode $m(X)$ into a vector $z \in \mathbb{C}^{N/2}$ containing the computed outputs.

To understand how this is done, we will see how we can encode and decode complex values into polynomials. To do so, we will use the canonical embedding $\sigma$ of $\mathbb{R}[X]/(X^N + 1)$ into $\mathbb{C}^{N}$. For any polynomial $m(X) \in  \mathbb{R}[X]/(X^N + 1)$, we have $\sigma(m) = (m(\xi_m^i))_{i = 1,3...n} \in \mathbb{C}^N$, i.e. $\sigma$ simply evaluates a polynomial $m$, on the roots of $X^N + 1$.

We can do the opposite operation which takes a vector $z \in \mathbb{C}^N$, and outputs a polynomial $m(X)$.

We will see an example now.

Let $M = 8$, $N = \frac{M}{2}= 4$, $\Phi_M(X) = X^4 + 1$, and $\omega = e^{\frac{2 i \pi}{8}} = e^{\frac{i \pi}{4}}$.
Our goal here is to encode the following vectors : $[1, 2, 3, 4]$ and $[-1, -2, -3, -4]$, decode them, add and multiply their polynomial and decode it.

![title](images/roots.PNG)
<center>Roots of $X^4 + 1$ (source : <a href="https://heat-project.eu/School/Chris%20Peikert/slides-heat2.pdf">Cryptography from Rings, HEAT summer school 2016)</a></center>

As we saw, in order to decode a polynomial, we simply need to evaluate it on powers of an $M$-th root of unity. Here we choose $\xi = \omega = e^{\frac{i \pi}{4}}$.

Once we have $\xi$ and $M$, we can define both $\sigma$ and its inverse $\sigma^{-1}$, respectively the decoding and the encoding.

Computing the output of $\sigma$ is easy, but it is trickier to compute $\sigma^{-1}$. Indeed, for any $z \in \mathbb{C}^n$, we must find a polynomial $m$ such that for all i=

This becomes simply a linear equation to solve of the form $Ax=b$, with $A$ the [Vandermonde matrix](https://en.wikipedia.org/wiki/Vandermonde_matrix), and $b = z$ the vector we want to encode into a polynomial.

Now that we know how to compute both $\sigma$ and $\sigma^{-1}$, let's implement it !

In [1]:
import numpy as np

# First we set the parameters
M = 8
N = M //2

# We set xi, which will be used in our computations
xi = np.exp(2 * np.pi * 1j / M)
xi

(0.7071067811865476+0.7071067811865475j)

In [2]:
def vandermonde(xi: np.complex128, M: int):
    """Computes the Vandermonde matrix from a m-th root of unity."""
    
    N = M //2
    matrix = []
    # We will generate each row of the matrix
    for i in range(N):
        # For each row we select a different root
        root = xi ** (2 * i + 1)
        row = []
        
        # Then we store its powers
        for j in range(N):
            row.append(root ** j)
        matrix.append(row)
    return matrix

In [3]:
from numpy.polynomial import Polynomial as P

def sigma_inverse(b, xi, M):
    """Encodes the vector b in a polynomial using an M-th root of unity."""
    
    # First we create the Vandermonde matrix
    A = vandermonde(xi, M)
    
    # Then we solve the system
    coeffs = np.linalg.solve(A, b)
    
    # Finally we output the polynomial
    p = P(coeffs)
    return p

def sigma(p, xi, M):
    """Decodes a polynomial by applying it to the M-th roots of unity."""
    
    outputs = []
    N = M //2
    
    # We simply apply the polynomial on the roots
    for i in range(N):
        root = xi ** (2 * i + 1)
        output = p(root)
        outputs.append(output)
    return np.array(outputs)

Now we can start experimenting with real values, let's first encode a vector and see how it is encoded.

In [4]:
b = np.array([1, 2, 3, 4])
b

array([1, 2, 3, 4])

In [5]:
p = sigma_inverse(b, xi, M)
p

Polynomial([ 2.50000000e+00+4.44089210e-16j, -4.99600361e-16+7.07106781e-01j,
       -3.46944695e-16+5.00000000e-01j, -8.32667268e-16+7.07106781e-01j], domain=[-1,  1], window=[-1,  1])

Let's see now how we can extract the vector we had initially from the polynomial: 

In [6]:
b_reconstructed = sigma(p, xi, M)
b_reconstructed

array([1.-1.11022302e-16j, 2.-4.71844785e-16j, 3.+2.77555756e-17j,
       4.+2.22044605e-16j])

We can see that the reconstruction and the initial vector are very close.

In [7]:
np.linalg.norm(b_reconstructed - b)

6.944442800358888e-16

We can now start to encode several vectors, and see how we can perform homomorphic operations on them and decode it.

In [8]:
m1 = np.array([1, 2, 3, 4])
m2 = np.array([1, -2, 3, -4])

In [9]:
p1 = sigma_inverse(m1, xi, M)
p2 = sigma_inverse(m2, xi, M)

Because when doing multiplication we might have terms whose degree is higher than $N$, we will need to do a modulo operation using $X^N + 1$.

In [10]:
from numpy.polynomial import polynomial as poly

def polymul(p1,p2,poly_modulo):
    p12 = p1 * p2
    return P(poly.polydiv(p12.coef, poly_modulo.coef)[1])

In [11]:
p_add = p1 + p2
p_add

Polynomial([ 2.00000000e+00+1.11022302e-16j, -7.07106781e-01+7.07106781e-01j,
        2.10942375e-15-2.00000000e+00j,  7.07106781e-01+7.07106781e-01j], domain=[-1.,  1.], window=[-1.,  1.])

Here as expected, we see that p1 + p2 decodes correctly to $[2, 0, 6, 0]$.

In [12]:
sigma(p_add, xi, M)

array([2.0000000e+00+3.25176795e-17j, 4.4408921e-16-4.44089210e-16j,
       6.0000000e+00+1.11022302e-16j, 4.4408921e-16+3.33066907e-16j])

To perform multiplication, we first need to define the polynomial modulus which we will use.

In [13]:
poly_modulo = P([1,0,0,0,1])
poly_modulo

Polynomial([1., 0., 0., 0., 1.], domain=[-1,  1], window=[-1,  1])

Now we can perform multiplication.

In [14]:
p_mult = polymul(p1, p2, poly_modulo)
p_mult

Polynomial([-2.50000000e+00-3.75394160e-15j, -3.53553391e+00-7.07106781e-01j,
        1.04958210e-14-7.50000000e+00j,  3.53553391e+00-7.07106781e-01j], domain=[-1,  1], window=[-1,  1])

Finally if we decode it, we can see that we have the expected result.

In [15]:
sigma(p_mult, xi, M)

array([  1.-8.67361738e-16j,  -4.+6.86950496e-16j,   9.+6.86950496e-16j,
       -16.-9.08301212e-15j])

In [16]:
p = P([1,2,1,2,1])
p

Polynomial([1., 2., 1., 2., 1.], domain=[-1,  1], window=[-1,  1])

In [17]:
xi

(0.7071067811865476+0.7071067811865475j)

In [18]:
p(xi), p(xi ** 7)

((8.881784197001252e-16+3.8284271247461907j),
 (-5.10702591327572e-15-3.8284271247461854j))

In [19]:
p(xi ** 3), p(xi ** 5)

((7.771561172376096e-16+1.8284271247461903j),
 (-1.7763568394002505e-15-1.8284271247461892j))

In [20]:
xi ** 7

(0.707106781186547-0.707106781186548j)

## CKKS test

In [21]:
b = [p(xi ** (2 * i + 1)) for i in range(N)]

In [24]:
R_basis = []
for i in range(N):
    mono = np.zeros(N)
    mono[i] = 1
    R_basis.append(P(mono))

In [28]:
sigmoid_R_basis = [sigma(p, xi, M) for p in R_basis]

In [34]:
sigmoid_R_basis = np.array(sigmoid_R_basis).T

In [43]:
z = [1,1,1,1]

b = np.matmul(sigmoid_R_basis, z)
b

array([1.+2.41421356j, 1.+0.41421356j, 1.-0.41421356j, 1.-2.41421356j])

In [44]:
p = sigma_inverse(b, xi, M)
p

Polynomial([1.+2.22044605e-16j, 1.+0.00000000e+00j, 1.+2.77555756e-17j,
       1.+2.22044605e-16j], domain=[-1,  1], window=[-1,  1])

In [42]:
sigma(p, xi, M)

array([1.+2.41421356j, 1.+0.41421356j, 1.-0.41421356j, 1.-2.41421356j])

In [45]:
sigma_inverse(b, xi, M)(xi)

(1+2.414213562373095j)